PASO 1
CONEXION A LA API, VARIABLES DE CIUDAD, PAIS, IDIOMA Y UNIDADES
LEYENDO API KEY DESDE TXT

In [2]:
import requests
import pandas as pd
import json

city_name = 'Buenos Aires'
country_code = 'AR'
lang = 'es'
units = 'metric'
with open(r"C:\Users\Julito\Desktop\Data Engineering\CoderHouse\DE\1Entregable\api_key.txt",'r') as p:
    api_key = p.read()

response = requests.get(
    f'https://api.openweathermap.org/data/2.5/weather?q={city_name},{country_code}&appid={api_key}&lang={lang}&units={units}')
raw_weather_data = response.json()
raw_weather_data

{'coord': {'lon': -58.3772, 'lat': -34.6132},
 'weather': [{'id': 800,
   'main': 'Clear',
   'description': 'cielo claro',
   'icon': '01d'}],
 'base': 'stations',
 'main': {'temp': 19.61,
  'feels_like': 19.25,
  'temp_min': 19.32,
  'temp_max': 21.1,
  'pressure': 1019,
  'humidity': 62,
  'sea_level': 1019,
  'grnd_level': 1015},
 'visibility': 10000,
 'wind': {'speed': 6.76, 'deg': 105, 'gust': 7.4},
 'clouds': {'all': 1},
 'dt': 1697746052,
 'sys': {'type': 2,
  'id': 2008409,
  'country': 'AR',
  'sunrise': 1697706389,
  'sunset': 1697753413},
 'timezone': -10800,
 'id': 3435910,
 'name': 'Buenos Aires',
 'cod': 200}

In [3]:
#aca se nota que la info principal asi como otros valores vienen en un dict dentro del dict principal
raw_weather_data['main']

{'temp': 19.61,
 'feels_like': 19.25,
 'temp_min': 19.32,
 'temp_max': 21.1,
 'pressure': 1019,
 'humidity': 62,
 'sea_level': 1019,
 'grnd_level': 1015}

In [4]:
#para poder convertirlo en un dataframe uso entonces la normalizacion y posterior concatenacion
df = pd.json_normalize(raw_weather_data)
norm_weather_df = pd.json_normalize(df['weather'][0])
all_weather_df = pd.concat([df, norm_weather_df], axis=1)
#lo que ya devuelve un dataframe con el que es posible trabajar
all_weather_df

,weather,base,visibility,dt,timezone,id,name,cod,coord.lon,coord.lat,...,clouds.all,sys.type,sys.id,sys.country,sys.sunrise,sys.sunset,id,main,description,icon
0,"[{'id': 800, 'main': 'Clear', 'description': '...",stations,10000,1697746052,-10800,3435910,Buenos Aires,200,-58.3772,-34.6132,...,1,2,2008409,AR,1697706389,1697753413,800,Clear,cielo claro,01d


In [5]:
#revisando las columnas que quedan actualmente para trabajar
all_weather_df.columns

Index(['weather', 'base', 'visibility', 'dt', 'timezone', 'id', 'name', 'cod',
       'coord.lon', 'coord.lat', 'main.temp', 'main.feels_like',
       'main.temp_min', 'main.temp_max', 'main.pressure', 'main.humidity',
       'main.sea_level', 'main.grnd_level', 'wind.speed', 'wind.deg',
       'wind.gust', 'clouds.all', 'sys.type', 'sys.id', 'sys.country',
       'sys.sunrise', 'sys.sunset', 'id', 'main', 'description', 'icon'],
      dtype='object')

In [6]:
#en este paso se eliminan las columnas que no van a utilizarse
reduced_weather_df = all_weather_df.drop(['base', 'timezone', 'id', 'cod', 'sys.type', 'sys.id', 'id', 'icon'], axis=1)
reduced_weather_df

,weather,visibility,dt,name,coord.lon,coord.lat,main.temp,main.feels_like,main.temp_min,main.temp_max,...,main.grnd_level,wind.speed,wind.deg,wind.gust,clouds.all,sys.country,sys.sunrise,sys.sunset,main,description
0,"[{'id': 800, 'main': 'Clear', 'description': '...",10000,1697746052,Buenos Aires,-58.3772,-34.6132,19.61,19.25,19.32,21.1,...,1015,6.76,105,7.4,1,AR,1697706389,1697753413,Clear,cielo claro


In [7]:
#vista previa del resultado del paso anterior
reduced_weather_df

,weather,visibility,dt,name,coord.lon,coord.lat,main.temp,main.feels_like,main.temp_min,main.temp_max,...,main.grnd_level,wind.speed,wind.deg,wind.gust,clouds.all,sys.country,sys.sunrise,sys.sunset,main,description
0,"[{'id': 800, 'main': 'Clear', 'description': '...",10000,1697746052,Buenos Aires,-58.3772,-34.6132,19.61,19.25,19.32,21.1,...,1015,6.76,105,7.4,1,AR,1697706389,1697753413,Clear,cielo claro


In [8]:
#confirmacion de la cantidad y calidad de columnas
reduced_weather_df.columns

Index(['weather', 'visibility', 'dt', 'name', 'coord.lon', 'coord.lat',
       'main.temp', 'main.feels_like', 'main.temp_min', 'main.temp_max',
       'main.pressure', 'main.humidity', 'main.sea_level', 'main.grnd_level',
       'wind.speed', 'wind.deg', 'wind.gust', 'clouds.all', 'sys.country',
       'sys.sunrise', 'sys.sunset', 'main', 'description'],
      dtype='object')

In [9]:
#ordenar las columnas en este paso me resulta mas sencillo que hacerlo posteriormente en la carga
sorted_weather_df = reduced_weather_df[['dt','name','sys.country','coord.lon','coord.lat','main','description','main.temp','main.feels_like','main.temp_min','main.temp_max','main.pressure','main.humidity','wind.speed','wind.deg','clouds.all','sys.sunrise','sys.sunset']]
sorted_weather_df

,dt,name,sys.country,coord.lon,coord.lat,main,description,main.temp,main.feels_like,main.temp_min,main.temp_max,main.pressure,main.humidity,wind.speed,wind.deg,clouds.all,sys.sunrise,sys.sunset
0,1697746052,Buenos Aires,AR,-58.3772,-34.6132,Clear,cielo claro,19.61,19.25,19.32,21.1,1019,62,6.76,105,1,1697706389,1697753413


In [10]:
#los 3 campos de tipo date time venian en formato UNIX, entonces usando to_datetime, los llevo primero a GMT 0
sorted_weather_df['dt'] = pd.to_datetime(sorted_weather_df['dt'], unit='s')
sorted_weather_df['sys.sunrise'] = pd.to_datetime(sorted_weather_df['sys.sunrise'], unit='s')
sorted_weather_df['sys.sunset'] = pd.to_datetime(sorted_weather_df['sys.sunset'], unit='s')
#y veo el resultado
sorted_weather_df

C:\Users\Julito\AppData\Local\Temp\ipykernel_7556\4229100300.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sorted_weather_df['dt'] = pd.to_datetime(sorted_weather_df['dt'], unit='s')
C:\Users\Julito\AppData\Local\Temp\ipykernel_7556\4229100300.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sorted_weather_df['sys.sunrise'] = pd.to_datetime(sorted_weather_df['sys.sunrise'], unit='s')
C:\Users\Julito\AppData\Local\Temp\ipykernel_7556\4229100300.py:4: SettingWithCopyWarning: 
A value is trying to be 

,dt,name,sys.country,coord.lon,coord.lat,main,description,main.temp,main.feels_like,main.temp_min,main.temp_max,main.pressure,main.humidity,wind.speed,wind.deg,clouds.all,sys.sunrise,sys.sunset
0,2023-10-19 20:07:32,Buenos Aires,AR,-58.3772,-34.6132,Clear,cielo claro,19.61,19.25,19.32,21.1,1019,62,6.76,105,1,2023-10-19 09:06:29,2023-10-19 22:10:13


In [11]:
from datetime import timedelta
#importo timedelta para poder restar 3 horas, y que asi se corresponda a la zona horaria donde estoy (GMT -3)
sorted_weather_df['dt'] = sorted_weather_df['dt'] - timedelta(hours=3)
sorted_weather_df['sys.sunrise'] = sorted_weather_df['sys.sunrise'] - timedelta(hours=3)
sorted_weather_df['sys.sunset'] = sorted_weather_df['sys.sunset'] - timedelta(hours=3)

C:\Users\Julito\AppData\Local\Temp\ipykernel_7556\3430822645.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sorted_weather_df['dt'] = sorted_weather_df['dt'] - timedelta(hours=3)
C:\Users\Julito\AppData\Local\Temp\ipykernel_7556\3430822645.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sorted_weather_df['sys.sunrise'] = sorted_weather_df['sys.sunrise'] - timedelta(hours=3)
C:\Users\Julito\AppData\Local\Temp\ipykernel_7556\3430822645.py:5: SettingWithCopyWarning: 
A value is trying to be set on a c

In [12]:
#este ya es el df final, del que podria exportarse a la tabla en RedShift
sorted_weather_df

,dt,name,sys.country,coord.lon,coord.lat,main,description,main.temp,main.feels_like,main.temp_min,main.temp_max,main.pressure,main.humidity,wind.speed,wind.deg,clouds.all,sys.sunrise,sys.sunset
0,2023-10-19 17:07:32,Buenos Aires,AR,-58.3772,-34.6132,Clear,cielo claro,19.61,19.25,19.32,21.1,1019,62,6.76,105,1,2023-10-19 06:06:29,2023-10-19 19:10:13


In [16]:
#mas facil poder leerlo con el tipo de dato y facilitar la creacion de la query
sorted_weather_df.dtypes

dt                 datetime64[ns]
name                       object
sys.country                object
coord.lon                 float64
coord.lat                 float64
main                       object
description                object
main.temp                 float64
main.feels_like           float64
main.temp_min             float64
main.temp_max             float64
main.pressure               int64
main.humidity               int64
wind.speed                float64
wind.deg                    int64
clouds.all                  int64
sys.sunrise        datetime64[ns]
sys.sunset         datetime64[ns]
dtype: object